In [ ]:
!pip install MFLES --upgrade

In [ ]:
!pip install statsforecast

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from statsforecast.models import MSTL, AutoETS
from MFLES.Forecaster import MFLES
import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:25: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Hourly Dataset

In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/Hourly-train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/Hourly-test.csv')

train_df.index = train_df['V1']
train_df = train_df.drop('V1', axis = 1)
test_df.index = test_df['V1']
test_df = test_df.drop('V1', axis = 1)

In [ ]:
smapes = []
naive_smape = []
mstl_smape = []
ets_smape = []
j = tqdm(range(len(train_df)))

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

for row in j:
    y = train_df.iloc[row, :].dropna()
    snaive = y.shift(1).dropna()
    snaive = snaive.values
    y_test = test_df.iloc[row, :].dropna().values
    j.set_description(f'MFLES:{np.mean(smapes)}, MSTL:{np.mean(mstl_smape)}, ETS:{np.mean(ets_smape)}, Naive:{np.mean(naive_smape)}')
    mfles = MFLES()
    fitted = mfles.fit(y.values, seasonal_period=[24, 168])
    predicted = mfles.predict(len(y_test))
    smapes.append(smape(y_test, predicted.clip(min=0)))
    naive_smape.append(smape(y_test, np.tile(y.iloc[-1], len(y_test))))

    mstl_model = MSTL(season_length=[24, 168])
    mstl_model = mstl_model.fit(y=y)
    fitted = mstl_model.predict_in_sample()['fitted']
    predicted = mstl_model.predict(h=len(y_test))['mean']
    mstl_smape.append(smape(y_test, predicted.clip(min=0)))

    ets_model = AutoETS(season_length=24, model='ZZZ')
    ets_model = ets_model.fit(y=y.values)
    fitted = ets_model.predict_in_sample()['fitted']
    predicted = ets_model.predict(h=len(y_test))['mean']
    ets_smape.append(smape(y_test, predicted.clip(min=0)))
print(f'MFLES {np.mean(smapes)}')
print(f'MSTL {np.mean(mstl_smape)}')
print(f'ETS {np.nanmean(ets_smape)}')
print(f'Naive {np.mean(naive_smape)}')

  0%|          | 0/414 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
nan MFLES:nan, MSTL:nan, ETS:nan, Naive:nan:   0%|          | 0/414 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/MFLES/Model.py:29: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, C))
  tmp = np.dot(X[:, ii], R)
nan MFLES:nan, MSTL:nan, ETS:nan, Naive:nan:   0%|          | 1/414 [00:36<4:09:09, 36.20s/it]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: inva

MFLES 13.394827502574199
MSTL 16.896619120361535
ETS 17.191376329339505
Naive 43.002986836424824


In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/Yearly-train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/Yearly-test.csv')

train_df.index = train_df['V1']
train_df = train_df.drop('V1', axis = 1)
test_df.index = test_df['V1']
test_df = test_df.drop('V1', axis = 1)

In [ ]:
smapes = []
naive_smape = []
naive_mase = []
model_mase = []
mstl_smape = []
ets_smape = []
sm_smape = []
mases = []
coverage_scores = []
j = tqdm(range(len(train_df)))

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

for row in j:
    y = train_df.iloc[row, :].dropna()
    snaive = y.shift(1).dropna()
    snaive = snaive.values
    y_test = test_df.iloc[row, :].dropna().values
    j.set_description(f'{np.mean(coverage_scores)} MFLES:{np.mean(smapes)}, MSTL:{np.mean(mstl_smape)}, ETS:{np.mean(ets_smape)}, Naive:{np.mean(naive_smape)}')
    mfles = MFLES()
    fitted = mfles.fit(y.values)
    predicted = mfles.predict(len(y_test))
    smapes.append(smape(y_test, predicted.clip(min=0)))
    naive_smape.append(smape(y_test, np.tile(y.iloc[-1], len(y_test))))

    # mstl_model = MSTL(season_length=[24, 168])
    # mstl_model = mstl_model.fit(y=y)
    # fitted = mstl_model.predict_in_sample()['fitted']
    # predicted = mstl_model.predict(h=len(y_test))['mean']
    # mstl_smape.append(smape(y_test, predicted.clip(min=0)))

    # ets_model = AutoETS(model='ZZZ')
    # ets_model = ets_model.fit(y=y.values)
    # fitted = ets_model.predict_in_sample()['fitted']
    # predicted = ets_model.predict(h=len(y_test))['mean']
    # ets_smape.append(smape(y_test, predicted.clip(min=0)))
print(f'MFLES {np.mean(smapes)}')
print(f'MSTL {np.mean(mstl_smape)}')
print(f'ETS {np.nanmean(ets_smape)}')
print(f'Naive {np.mean(naive_smape)}')

  0%|          | 0/23000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
nan MFLES:nan, MSTL:nan, ETS:nan, Naive:nan:   0%|          | 0/23000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/MFLES/Model.py:29: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, C))
  tmp = np.dot(X[:, ii], R)
nan MFLES:nan, MSTL:nan, ETS:nan, Naive:nan:   0%|          | 1/23000 [00:28<182:05:29, 28.50s/it]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarni

MFLES 14.985593384030988
MSTL nan
ETS nan
Naive 16.34217348811709


<ipython-input-5-e40a7753de48>:40: RuntimeWarning: Mean of empty slice
  print(f'ETS {np.nanmean(ets_smape)}')


In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/Monthly-train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/Monthly-test.csv')

train_df.index = train_df['V1']
train_df = train_df.drop('V1', axis = 1)
test_df.index = test_df['V1']
test_df = test_df.drop('V1', axis = 1)

In [ ]:
smapes = []
naive_smape = []
naive_mase = []
model_mase = []
mstl_smape = []
ets_smape = []
sm_smape = []
mases = []
coverage_scores = []
j = tqdm(range(len(train_df)))

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

for row in j:
    y = train_df.iloc[row, :].dropna()
    snaive = y.shift(1).dropna()
    snaive = snaive.values
    y_test = test_df.iloc[row, :].dropna().values
    j.set_description(f'{np.mean(coverage_scores)} MFLES:{np.mean(smapes)}, MSTL:{np.mean(mstl_smape)}, ETS:{np.mean(ets_smape)}, Naive:{np.mean(naive_smape)}')
    mfles = MFLES()
    fitted = mfles.fit(y.values, seasonal_period=[12])
    predicted = mfles.predict(len(y_test))
    smapes.append(smape(y_test, predicted.clip(min=0)))
    naive_smape.append(smape(y_test, np.tile(y.iloc[-1], len(y_test))))

    # mstl_model = MSTL(season_length=[12])
    # mstl_model = mstl_model.fit(y=y)
    # fitted = mstl_model.predict_in_sample()['fitted']
    # predicted = mstl_model.predict(h=len(y_test))['mean']
    # mstl_smape.append(smape(y_test, predicted.clip(min=0)))

    # ets_model = AutoETS(season_length=12, model='ZZZ')
    # ets_model = ets_model.fit(y=y.values)
    # fitted = ets_model.predict_in_sample()['fitted']
    # predicted = ets_model.predict(h=len(y_test))['mean']
    # ets_smape.append(smape(y_test, predicted.clip(min=0)))
print(f'MFLES {np.mean(smapes)}')
print(f'MSTL {np.mean(mstl_smape)}')
print(f'ETS {np.nanmean(ets_smape)}')
print(f'Naive {np.mean(naive_smape)}')

  0%|          | 0/48000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
nan MFLES:nan, MSTL:nan, ETS:nan, Naive:nan:   0%|          | 1/48000 [00:05<71:55:53,  5.39s/it]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
nan MFLES:13.303045733106968, MSTL:nan, ETS:nan, Naive:15.256743936162751: 100%|██████████| 48000/48000 [14:20<00:00, 55.78it/s]
<ipython-input-7-b065ded8d9b5>:40: RuntimeWarning: Mean of empty slice
  print(f'ET

MFLES 13.302832302129898
MSTL nan
ETS nan
Naive 15.256483534162772


In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/Weekly-train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/Weekly-test.csv')

train_df.index = train_df['V1']
train_df = train_df.drop('V1', axis = 1)
test_df.index = test_df['V1']
test_df = test_df.drop('V1', axis = 1)

In [ ]:
smapes = []
naive_smape = []
naive_mase = []
model_mase = []
mstl_smape = []
ets_smape = []
sm_smape = []
mases = []
coverage_scores = []
j = tqdm(range(len(train_df)))

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

for row in j:
    y = train_df.iloc[row, :].dropna()
    snaive = y.shift(1).dropna()
    snaive = snaive.values
    y_test = test_df.iloc[row, :].dropna().values
    j.set_description(f'{np.mean(coverage_scores)} MFLES:{np.mean(smapes)}, MSTL:{np.mean(mstl_smape)}, ETS:{np.mean(ets_smape)}, Naive:{np.mean(naive_smape)}')
    mfles = MFLES()
    fitted = mfles.fit(y.values, seasonal_period=[52])
    predicted = mfles.predict(len(y_test))
    smapes.append(smape(y_test, predicted.clip(min=0)))
    naive_smape.append(smape(y_test, np.tile(y.iloc[-1], len(y_test))))

    mstl_model = MSTL(season_length=[52])
    mstl_model = mstl_model.fit(y=y)
    fitted = mstl_model.predict_in_sample()['fitted']
    predicted = mstl_model.predict(h=len(y_test))['mean']
    mstl_smape.append(smape(y_test, predicted.clip(min=0)))

    ets_model = AutoETS(season_length=52, model='ZZZ')
    ets_model = ets_model.fit(y=y.values)
    fitted = ets_model.predict_in_sample()['fitted']
    predicted = ets_model.predict(h=len(y_test))['mean']
    ets_smape.append(smape(y_test, predicted.clip(min=0)))
print(f'MFLES {np.mean(smapes)}')
print(f'MSTL {np.mean(mstl_smape)}')
print(f'ETS {np.nanmean(ets_smape)}')
print(f'Naive {np.mean(naive_smape)}')

  0%|          | 0/359 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
nan MFLES:nan, MSTL:nan, ETS:nan, Naive:nan:   0%|          | 1/359 [00:33<3:18:02, 33.19s/it]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
nan MFLES:1.6640929430192204, MSTL:5.77026381077837, ETS:1.7376219041474052, Naive:2.0568427004644523:   1%|          | 2/359 [00:40<1:46:52, 17.96s/it]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py

MFLES 8.601028297380996
MSTL 10.435823721950934
ETS 8.636611002006651
Naive 9.161286913982


In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/Quarterly-train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/Quarterly-test.csv')

train_df.index = train_df['V1']
train_df = train_df.drop('V1', axis = 1)
test_df.index = test_df['V1']
test_df = test_df.drop('V1', axis = 1)

In [ ]:
smapes = []
naive_smape = []
naive_mase = []
model_mase = []
mstl_smape = []
ets_smape = []
sm_smape = []
mases = []
coverage_scores = []
j = tqdm(range(len(train_df)))

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

for row in j:
    y = train_df.iloc[row, :].dropna()
    snaive = y.shift(1).dropna()
    snaive = snaive.values
    y_test = test_df.iloc[row, :].dropna().values
    j.set_description(f'{np.mean(coverage_scores)} MFLES:{np.mean(smapes)}, MSTL:{np.mean(mstl_smape)}, ETS:{np.mean(ets_smape)}, Naive:{np.mean(naive_smape)}')
    mfles = MFLES()
    fitted = mfles.fit(y.values, seasonal_period=[4])
    predicted = mfles.predict(len(y_test))
    smapes.append(smape(y_test, predicted.clip(min=0)))
    naive_smape.append(smape(y_test, np.tile(y.iloc[-1], len(y_test))))

    # mstl_model = MSTL(season_length=[52])
    # mstl_model = mstl_model.fit(y=y)
    # fitted = mstl_model.predict_in_sample()['fitted']
    # predicted = mstl_model.predict(h=len(y_test))['mean']
    # mstl_smape.append(smape(y_test, predicted.clip(min=0)))

    # ets_model = AutoETS(season_length=52, model='ZZZ')
    # ets_model = ets_model.fit(y=y.values)
    # fitted = ets_model.predict_in_sample()['fitted']
    # predicted = ets_model.predict(h=len(y_test))['mean']
    # ets_smape.append(smape(y_test, predicted.clip(min=0)))
print(f'MFLES {np.mean(smapes)}')
print(f'MSTL {np.mean(mstl_smape)}')
print(f'ETS {np.nanmean(ets_smape)}')
print(f'Naive {np.mean(naive_smape)}')

  0%|          | 0/24000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
nan MFLES:10.880815007549614, MSTL:nan, ETS:nan, Naive:11.610448045602293: 100%|██████████| 24000/24000 [03:01<00:00, 132.27it/s]

MFLES 10.880701070852508
MSTL nan
ETS nan
Naive 11.610267157878681



<ipython-input-9-5100b229d118>:40: RuntimeWarning: Mean of empty slice
  print(f'ETS {np.nanmean(ets_smape)}')


In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/Daily-train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/Daily-test.csv')

train_df.index = train_df['V1']
train_df = train_df.drop('V1', axis = 1)
test_df.index = test_df['V1']
test_df = test_df.drop('V1', axis = 1)

In [ ]:
smapes = []
naive_smape = []
naive_mase = []
model_mase = []
mstl_smape = []
ets_smape = []
sm_smape = []
mases = []
coverage_scores = []
j = tqdm(range(len(train_df)))

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

for row in j:
    y = train_df.iloc[row, :].dropna()
    snaive = y.shift(1).dropna()
    snaive = snaive.values
    y_test = test_df.iloc[row, :].dropna().values
    j.set_description(f'{np.mean(coverage_scores)} MFLES:{np.mean(smapes)}, MSTL:{np.mean(mstl_smape)}, ETS:{np.mean(ets_smape)}, Naive:{np.mean(naive_smape)}')
    mfles = MFLES()
    fitted = mfles.fit(y.values, seasonal_period=[7])
    predicted = mfles.predict(len(y_test))
    smapes.append(smape(y_test, predicted.clip(min=0)))
    naive_smape.append(smape(y_test, np.tile(y.iloc[-1], len(y_test))))

    # mstl_model = MSTL(season_length=[52])
    # mstl_model = mstl_model.fit(y=y)
    # fitted = mstl_model.predict_in_sample()['fitted']
    # predicted = mstl_model.predict(h=len(y_test))['mean']
    # mstl_smape.append(smape(y_test, predicted.clip(min=0)))

    # ets_model = AutoETS(season_length=52, model='ZZZ')
    # ets_model = ets_model.fit(y=y.values)
    # fitted = ets_model.predict_in_sample()['fitted']
    # predicted = ets_model.predict(h=len(y_test))['mean']
    # ets_smape.append(smape(y_test, predicted.clip(min=0)))
print(f'MFLES {np.mean(smapes)}')
print(f'MSTL {np.mean(mstl_smape)}')
print(f'ETS {np.nanmean(ets_smape)}')
print(f'Naive {np.mean(naive_smape)}')

  0%|          | 0/4227 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
nan MFLES:3.1470325308405624, MSTL:nan, ETS:nan, Naive:3.0453675912293523: 100%|██████████| 4227/4227 [21:51<00:00,  3.22it/s]

MFLES 3.1469120939941204
MSTL nan
ETS nan
Naive 3.0452517800636607



<ipython-input-13-693566882334>:40: RuntimeWarning: Mean of empty slice
  print(f'ETS {np.nanmean(ets_smape)}')
